In [6]:
#https://zhuanlan.zhihu.com/p/39599265
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def layer_sizes(X, Y):
    n_x = X.shape[0] # size of input layer
    w_x = X.shape[1]
    n_h = 4 # size of hidden layer
    n_y = Y.shape[0] # size of output layer
    return (n_x, n_h, n_y,w_x)

def initialize_parameters(n_x, n_h, n_y,w_x):
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, w_x))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, w_x)) 
   
    assert (W1.shape == (n_h, n_x))    
    assert (b1.shape == (n_h, w_x))    
    assert (W2.shape == (n_y, n_h))    
    assert (b2.shape == (n_y, w_x))
    parameters = {"W1": W1, 
                  "b1": b1,                 
                  "W2": W2,                  
                  "b2": b2}   
                   
    return parameters

def forward_propagation(X, parameters):
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']    
    # Implement Forward Propagation to calculate A2 (probabilities)
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, Z1) + b2
    A2 = sigmoid(Z2)    
    print("A2.shape=",A2.shape," X.shape=",X.shape)
    #assert(A2.shape == (1, X.shape[1]))
    assert(A2.shape == X.shape)
    cache = {"Z1": Z1,                   
             "A1": A1,                   
             "Z2": Z2,                  
             "A2": A2}    
    return A2, cache

def compute_cost(A2, Y, parameters):
    print("Y.shape=",Y.shape)
    print("A2.shape=",A2.shape)
    #m = Y.shape[1] # number of example
    m = Y.shape[0] # number of example
    # Compute the cross-entropy cost
    logprobs = np.multiply(np.log(A2),Y) + np.multiply(np.log(1-A2), 1-Y)
    cost = -1/m * np.sum(logprobs)
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect.
    assert(isinstance(cost, float))    
    return cost

def backward_propagation(parameters, cache, X, Y):
    m = X.shape[1]    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    W1 = parameters['W1']
    W2 = parameters['W2']    
    # Retrieve also A1 and A2 from dictionary "cache".
    A1 = cache['A1']
    A2 = cache['A2']    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    dZ2 = A2-Y
    dW2 = 1/m * np.dot(dZ2, A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2)*(1-np.power(A1, 2))
    dW1 = 1/m * np.dot(dZ1, X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)
    grads = {"dW1": dW1,
             "db1": db1,                      
             "dW2": dW2,             
             "db2": db2}   
    return grads

def update_parameters(parameters, grads, learning_rate = 1.2):
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']    
    # Retrieve each gradient from the dictionary "grads"
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']    
    # Update rule for each parameter
    W1 -= dW1 * learning_rate
    b1 -= db1 * learning_rate
    W2 -= dW2 * learning_rate
    b2 -= db2 * learning_rate
    parameters = {"W1": W1, 
                  "b1": b1,            
                  "W2": W2,   
                  "b2": b2}    
    return parameters

def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    w_x = layer_sizes(X, Y)[3]
    # Initialize parameters, then retrieve W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    parameters = initialize_parameters(n_x, n_h, n_y,w_x)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']    
    # Loop (gradient descent)
    for i in range(0, num_iterations):        
    # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, Y, parameters)        
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)        
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads, learning_rate=1.2)        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:            
            print ("Cost after iteration %i: %f" %(i, cost))    
            
    return parameters

In [7]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
ylen = diabetes.target.shape
Y = np.zeros(ylen)
Y[diabetes.target>=np.mean(diabetes.target)]=1
print(X.shape,Y.shape)
from sklearn.cross_validation import train_test_split
#x为数据集的feature熟悉，y为label.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)
print("X_train.shape=",X_train.shape,"Y_train.shape=",Y_train.shape,";X_test.shape=",X_test.shape,"Y_test.shape=",Y_test.shape)

(442, 10) (442,)
X_train.shape= (309, 10) Y_train.shape= (309,) ;X_test.shape= (133, 10) Y_test.shape= (133,)


In [8]:
d = nn_model(X_train, Y_train, 4, num_iterations = 10000,  print_cost = True) 

A2.shape= (309, 10)  X.shape= (309, 10)
Y.shape= (309,)
A2.shape= (309, 10)


ValueError: operands could not be broadcast together with shapes (309,10) (309,) 

In [ ]:
print("")